## Tranforming and gridding JULES data

Using vegetation cover from JULES: this first requires the data to be transformed from .csv to .nc files. I've adaptedt the jules library to work with python3 (hopefully!).python2

In [1]:
%load_ext autoreload
%autoreload

import sys
sys.path.append('../')

import warnings
warnings.filterwarnings('ignore')

import os
from   io     import StringIO
import numpy  as np
import pandas as pd
import csv

import iris
import matplotlib.pyplot as plt
import iris.plot as iplt
import iris.quickplot as qplt
import iris.coord_categorisation
import cftime
import nc_time_axis
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs
from   libs.plot_maps    import *
from libs import jules3

In [4]:
# Location of juels data and core name of the file
dir = '../data/jules/'
file = 'JULES-ES.1p0.vn5.4.50.GSWP3.365.S3.Monthly.'

# Where it will be saved
outfile = '../data/UKESM/retrieved_codes/2000-2009/'


aList = []
years = range(2000,2010)

# For a single year
# df1 = jules.load(dir + file + '2000' + '.nc', 'Fractional cover of each surface type')

for y in years:
    df = jules.load(dir + file + str(y) + '.nc', 'Fractional cover of each surface type')
    print(y)
    aList.append(df[0])   

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009


Merging the list together.

In [5]:
cube_list = iris.cube.CubeList(aList)
cubes = cube_list.concatenate_cube()

print(cubes.coord('type'))

DimCoord(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]), standard_name=None, units=Unit('1'), long_name='type')


Need to add a coord_system to jules data, and then regrid it

In [6]:
# Template
dir_m = "../data/UKESM/retrieved_codes/2000-2009/"
file_m = 'vegcover' + "2000-2009.nc"

# And load it in:
mask = iris.load_cube(dir_m + file_m)

cubes.coord('latitude').coord_system = mask.coord('latitude').coord_system
cubes.coord('longitude').coord_system = mask.coord('longitude').coord_system
regrid_cube = cubes.regrid(mask, iris.analysis.Linear(extrapolation_mode ='nan'))

Extracting vegcover and treecover. This can easily be adapted to also extract pasture and cropland, if the ``var_type`` number of the different veg fraction is known.

In [14]:
tree = list(range(5))
veg = list(range(13))
var_name = ['treeCover', 'vegcover']
var_type = [tree, veg]
print(var_type[0])

[0, 1, 2, 3, 4]


Reformatting the Jules data and saving it as a netcdf.

In [15]:
for n in range(len(var_name)):

    index = [regrid_cube.coord('type').points == x  for x in var_type[n]]

    # This combines all the boolean arrays together. True + False = True
    index = np.any(index, axis = 0)

    # Extracts just the layers we want and saves
    cube = regrid_cube[:,index]

    # For just using from July 2000 to Jun 2014, but we don't have 2014...
    # Just taking the off the first 6 months
    cube = cube[6:,:,:,:].collapsed(['type'], iris.analysis.SUM)
    
    cube_copy = mask.copy()
    cube_copy.data = cube.data
    
    out = outfile + 'jules_' + var_name[n] + str(years[0]) + '-' + str(years[len(years) - 1]) + '.nc'
    iris.save(cube_copy, out)